In [1]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

TRAIN_SLOPE = 2
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10
SEED = 2

# Utils

In [2]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    torch.manual_seed(0)
    np.random.seed(0)
    data = torch.cat((Y, X), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, 1:]
    Y = data[:, 0]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# CCP classes

In [3]:
class CCP:
    def __init__(self, x_dim, batch_size, funcs, scale):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable((batch_size, x_dim))
        self.xt = cp.Parameter((batch_size, x_dim))
        self.r = cp.Parameter((batch_size, x_dim))
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = cp.diag(self.x@(self.f_derivative(self.xt, self.w, self.b, self.slope).T))-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(cp.sum(target)), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.001 and cnt < 100:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b, slope):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        return torch.from_numpy(self.ccp(X))
        # return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

In [4]:
class DELTA():
    
    def __init__(self, x_dim, funcs, scale):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [5]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r, x_dim, scale):
    return (scale)*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)
    
def f_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones(x.shape[0]), (slope*score(x, w, b) + 1)]), 2, axis=0)

def g_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones((1, x.shape[0])), cp.reshape((slope*score(x, w, b) - 1), (1, x.shape[0]))]), 2, axis=0)

def c_batch(x, r, x_dim, scale):
    return (scale)*cp.square(cp.norm(x-r, 2, axis=1))

def f_derivative_batch(x, w, b, slope):
    nablas = 0.5*slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1))
    return cp.reshape(nablas, (nablas.shape[0], 1))@cp.reshape(w, (1, x.shape[1]))

# Model

In [6]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, batch_size, funcs, funcs_batch, train_slope, eval_slope, scale, strategic=False):
        torch.manual_seed(0)
        np.random.seed(0)

        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.batch_size = batch_size
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(1, dtype=torch.float64, requires_grad=True)))
#         self.w = torch.nn.parameter.Parameter(torch.zeros(x_dim, dtype=torch.float64, requires_grad=True))
#         self.b = torch.nn.parameter.Parameter(torch.zeros(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.ccp = CCP(x_dim, batch_size, funcs_batch, scale)
        self.delta = DELTA(x_dim, funcs, scale)
        self.ccp_time = 0
        self.total_time = 0

    def forward(self, X, evaluation=False):
        if self.strategic:
            if evaluation:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.eval_slope)
                self.ccp_time += time.time()-t1
                X_opt = XT
            else:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.train_slope)
                self.ccp_time += time.time()-t1
                F_DER = self.get_f_ders(XT, self.train_slope)
                X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should be equal to XT but we do it again for the gradients
            output = self.score(X_opt)
        else:
            output = self.score(X)        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        return self.ccp.optimize_X(X, self.w, self.b, slope)
    
    def normalize_weights(self):
        with torch.no_grad():
            norm = torch.sqrt(torch.sum(self.w**2) + self.b**2)
            self.w /= norm
            self.b /= norm

    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        # return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])
        nablas = 0.5*slope*((slope*self.score(XT) + 1)/torch.sqrt((slope*self.score(XT) + 1)**2 + 1))
        return torch.reshape(nablas, (len(nablas), 1))@torch.reshape(self.w, (1, len(self.w)))

    def calc_accuracy(self, Y, Y_pred):
        Y_pred = torch.sign(Y_pred)
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def evaluate(self, X, Y):      
        return self.calc_accuracy(Y, self.forward(X, evaluation=True))
    
    def loss(self, Y, Y_pred):
        return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
    
    def save_model(self, train_errors, val_errors, train_losses, val_losses, info, path, comment=None):
        if comment is not None:
            path += "/" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
                
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, path, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        test_dset = TensorDataset(Xval, Yval)
        test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=True)
        
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        
        best_val_error = 1
        consecutive_no_improvement = 0

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
#                 try:
                opt.zero_grad()
                Ybatch_pred = self.forward(Xbatch)
                l = self.loss(Ybatch, Ybatch_pred)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                with torch.no_grad():
                    e = self.calc_accuracy(Ybatch, Ybatch_pred)
                    train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | err: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1]), np.mean(train_errors[-1])))
                batch += 1
                if callback is not None:
                    callback()
#                 except:
#                     print("failed")

            with torch.no_grad():
                total_loss = 0
                total_error = 0
                batch = 0
                for Xbatch, Ybatch in test_loader:
#                     try:
                    Yval_pred = self.forward(Xbatch, evaluation=True)
                    val_loss = self.loss(Ybatch, Yval_pred).item()
                    total_loss += val_loss
                    val_error = 1-self.calc_accuracy(Ybatch, Yval_pred)
                    total_error += val_error
                    batch += 1
#                     except:
#                         print("failed")
                        
                avg_loss = total_loss/batch
                avg_error = total_error/batch
                val_losses.append(avg_loss)
                val_errors.append(avg_error)
                if avg_error < best_val_error:
                        consecutive_no_improvement = 0
                        best_val_error = avg_error
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\n".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], avg_loss, avg_error)
                        self.save_model(train_errors, val_errors, train_losses, val_losses, info, path, comment)
                        print("model saved!")

                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 100:
                        break
                
            t2 = time.time()
            if verbose:
                print("------------- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        
        self.total_time = time.time()-total_time
        print("training time: {} seconds".format(self.total_time)) 
        return train_errors, val_errors, train_losses, val_losses

In [7]:
def gen_sklearn_data(x_dim, N, informative_frac=1, shift_range=1, scale_range=1, noise_frac=0.01):
    torch.manual_seed(0)
    np.random.seed(0)
    n_informative = int(informative_frac*x_dim)
    n_redundant = x_dim - n_informative
    shift_arr = shift_range*np.random.randn(x_dim)
    scale_arr = scale_range*np.random.randn(x_dim)
    X, Y = make_classification(n_samples=N, n_features=x_dim, n_informative=n_informative, n_redundant=n_redundant,
                               flip_y=noise_frac, shift=shift_arr, scale=scale_arr, random_state=0)
    Y[Y == 0] = -1
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_spam_data():
    torch.manual_seed(0)
    np.random.seed(0)
    path = r"C:\Users\sagil\Desktop\nir_project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_card_fraud_data():
    torch.manual_seed(0)
    np.random.seed(0)
    df = pd.read_csv('C:/Users/sagil/Desktop/nir_project/card_fraud_dataset/creditcard.csv')

    rob_scaler = RobustScaler()

    df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
    df.drop(['Time','Amount'], axis=1, inplace=True)
    scaled_amount = df['scaled_amount']
    df.drop(['scaled_amount'], axis=1, inplace=True)
    df.insert(0, 'scaled_amount', scaled_amount)

    df["Class"].replace({1: -1, 0: 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    # amount of fraud classes 492 rows.
    fraud_df = df.loc[df['Class'] == -1]
    non_fraud_df = df.loc[df['Class'] == 1][:492]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['Class'].values
    X = df.drop('Class', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_credit_default_data():
    torch.manual_seed(0)
    np.random.seed(0)
    url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
    df = pd.read_csv(url)
    df["NoDefaultNextMonth"].replace({0: -1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    df = df.drop(['Married', 'Single', 'Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60'], axis = 1)

    fraud_df = df.loc[df["NoDefaultNextMonth"] == -1]
    non_fraud_df = df.loc[df["NoDefaultNextMonth"] == 1][:6636]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    
    scaler = StandardScaler()
    df.loc[:, df.columns != "NoDefaultNextMonth"] = scaler.fit_transform(df.drop("NoDefaultNextMonth", axis=1)) 
    Y, X = df.iloc[:, 0].values, df.iloc[:, 1:].values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_financial_distress_data():
    torch.manual_seed(0)
    np.random.seed(0)
    data = pd.read_csv("C:/Users/sagil/Desktop/nir_project/financial_distress_data/Financial Distress.csv")

    data = data[data.columns.drop(list(data.filter(regex='x80')))] # Since it is a categorical feature with 37 features.
    x_dim = len(data.columns) - 3
    data.drop(['Time'], axis=1, inplace=True)

    data_grouped = data.groupby(['Company']).last()

    scaler = StandardScaler()
    data_grouped.loc[:, data_grouped.columns != "Financial Distress"] = scaler.fit_transform(data_grouped.drop("Financial Distress", axis=1))

    # Shuffle dataframe rows
    data_grouped = data_grouped.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y, X = data_grouped.iloc[:, 0].values, data_grouped.iloc[:, 1:].values
    for y in range(0,len(Y)): # Coverting target variable from continuous to binary form
        if Y[y] < -0.5:
              Y[y] = -1
        else:
              Y[y] = 1
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [8]:
training_datas = []

# distress 
X, Y = load_financial_distress_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 24,
                        "name": "distress"})

# fraud dataset
X, Y = load_card_fraud_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%24
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%24
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 24, 
                        "name": "fraud"})


# credit data
X, Y = load_credit_default_data()
X, Y = X[:3000], Y[:3000]
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%64
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%64
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 64, 
                        "name": "credit"})

# spam dataset
X, Y = load_spam_data()
X, Y, Xval, Yval = split_data(X, Y, 0.25)
n = len(X)
effective_n = n - n%128
X, Y = X[:effective_n], Y[:effective_n]
n = len(Xval)
effective_n = n - n%128
Xval, Yval = Xval[:effective_n], Yval[:effective_n]
training_datas.append({"X": X,
                        "Y": Y,
                        "Xval": Xval,
                        "Yval": Yval,
                        "epochs": 7,
                        "batch_size": 128, 
                        "name": "spam"})

In [9]:
PATH = "C:/Users/sagil/Desktop/nir_project/models/real_dataset_full_runtimes"

for training_data in training_datas:
    path = PATH + "/" + training_data["name"]
    
    # load dataset
    X = training_data["X"]
    Y = training_data["Y"]
    Xval = training_data["Xval"]
    Yval = training_data["Yval"]
    
    # save dataset splits
    if not os.path.exists(path):
        os.makedirs(path)
    pd.DataFrame(X.numpy()).to_csv(path + '/X.csv')
    pd.DataFrame(Y.numpy()).to_csv(path + '/Y.csv')
    pd.DataFrame(Xval.numpy()).to_csv(path + '/Xval.csv')
    pd.DataFrame(Yval.numpy()).to_csv(path + '/Yval.csv')
    
    # training parameters
    x_dim = len(X[0])
    epochs = training_data["epochs"]
    batch_size = training_data["batch_size"]
    scale = 1
    
    funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
    funcs_batch = {"f": f_batch, "g": g_batch, "f_derivative": f_derivative_batch, "c": c_batch, "score": score}


    strategic_model = MyStrategicModel(x_dim, batch_size, funcs, funcs_batch, TRAIN_SLOPE, EVAL_SLOPE, scale=scale, strategic=True)
    strategic_model.fit(path, X, Y, Xval, Yval,
                opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-1)},
                batch_size=batch_size, epochs=epochs, verbose=True,
               comment="batched")
    
    runtimes = [strategic_model.total_time, strategic_model.ccp_time]     
    pd.DataFrame(np.array(runtimes)).to_csv(path + '/results.csv')

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


batch 001 / 013 | loss: 0.89757 | err: 0.33333
batch 002 / 013 | loss: 1.00498 | err: 0.41667
batch 003 / 013 | loss: 0.92932 | err: 0.38889
batch 004 / 013 | loss: 0.80200 | err: 0.33333
batch 005 / 013 | loss: 0.70121 | err: 0.29167
batch 006 / 013 | loss: 0.63705 | err: 0.26389
batch 007 / 013 | loss: 0.56297 | err: 0.23214
batch 008 / 013 | loss: 0.53042 | err: 0.20833
batch 009 / 013 | loss: 0.50730 | err: 0.19907
batch 010 / 013 | loss: 0.47833 | err: 0.18750
batch 011 / 013 | loss: 0.44445 | err: 0.17424
batch 012 / 013 | loss: 0.42530 | err: 0.16667
batch 013 / 013 | loss: 0.42113 | err: 0.16026
model saved!
------------- epoch 001 / 007 | time: 032 sec | loss: 0.23893 | err: 0.07292
batch 001 / 013 | loss: 0.55506 | err: 0.16667
batch 002 / 013 | loss: 0.38631 | err: 0.12500
batch 003 / 013 | loss: 0.32111 | err: 0.09722
batch 004 / 013 | loss: 0.32333 | err: 0.09375
batch 005 / 013 | loss: 0.30259 | err: 0.09167
batch 006 / 013 | loss: 0.30986 | err: 0.09722
batch 007 / 013 |

batch 008 / 030 | loss: 0.16007 | err: 0.07813
batch 009 / 030 | loss: 0.14297 | err: 0.06944
batch 010 / 030 | loss: 0.15341 | err: 0.07500
batch 011 / 030 | loss: 0.15485 | err: 0.07576
batch 012 / 030 | loss: 0.14211 | err: 0.06944
batch 013 / 030 | loss: 0.14404 | err: 0.07051
batch 014 / 030 | loss: 0.14971 | err: 0.07143
batch 015 / 030 | loss: 0.15095 | err: 0.07222
batch 016 / 030 | loss: 0.14152 | err: 0.06771
batch 017 / 030 | loss: 0.15119 | err: 0.06863
batch 018 / 030 | loss: 0.15710 | err: 0.07176
batch 019 / 030 | loss: 0.15345 | err: 0.07018
batch 020 / 030 | loss: 0.16697 | err: 0.07708
batch 021 / 030 | loss: 0.16695 | err: 0.07738
batch 022 / 030 | loss: 0.16340 | err: 0.07576
batch 023 / 030 | loss: 0.15630 | err: 0.07246
batch 024 / 030 | loss: 0.16195 | err: 0.07465
batch 025 / 030 | loss: 0.17257 | err: 0.08000
batch 026 / 030 | loss: 0.18151 | err: 0.08333
batch 027 / 030 | loss: 0.18085 | err: 0.08333
batch 028 / 030 | loss: 0.17449 | err: 0.08036
batch 029 / 0

batch 023 / 035 | loss: 0.87625 | err: 0.43954
batch 024 / 035 | loss: 0.86904 | err: 0.43555
batch 025 / 035 | loss: 0.87266 | err: 0.43688
batch 026 / 035 | loss: 0.86821 | err: 0.43510
batch 027 / 035 | loss: 0.86512 | err: 0.43345
batch 028 / 035 | loss: 0.85582 | err: 0.42801
batch 029 / 035 | loss: 0.85419 | err: 0.42619
batch 030 / 035 | loss: 0.84973 | err: 0.42396
batch 031 / 035 | loss: 0.84728 | err: 0.42288
batch 032 / 035 | loss: 0.84445 | err: 0.42188
batch 033 / 035 | loss: 0.85036 | err: 0.42566
batch 034 / 035 | loss: 0.85010 | err: 0.42647
batch 035 / 035 | loss: 0.85506 | err: 0.43125
model saved!
------------- epoch 001 / 007 | time: 069 sec | loss: 0.77012 | err: 0.34659
batch 001 / 035 | loss: 0.98854 | err: 0.54688
batch 002 / 035 | loss: 0.90913 | err: 0.46875
batch 003 / 035 | loss: 0.89608 | err: 0.45833
batch 004 / 035 | loss: 0.86837 | err: 0.44141
batch 005 / 035 | loss: 0.86784 | err: 0.44375
batch 006 / 035 | loss: 0.86234 | err: 0.44271
batch 007 / 035 |

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


batch 013 / 035 | loss: 0.82824 | err: 0.41827
batch 014 / 035 | loss: 0.83372 | err: 0.41964
batch 015 / 035 | loss: 0.81616 | err: 0.41042
batch 016 / 035 | loss: 0.82385 | err: 0.41406
batch 017 / 035 | loss: 0.81915 | err: 0.41176
batch 018 / 035 | loss: 0.82486 | err: 0.41667
batch 019 / 035 | loss: 0.81424 | err: 0.40954
batch 020 / 035 | loss: 0.81115 | err: 0.40781
batch 021 / 035 | loss: 0.80337 | err: 0.40476
batch 022 / 035 | loss: 0.80199 | err: 0.40412
batch 023 / 035 | loss: 0.80511 | err: 0.40625
batch 024 / 035 | loss: 0.80778 | err: 0.40820
batch 025 / 035 | loss: 0.81169 | err: 0.41063
batch 026 / 035 | loss: 0.81577 | err: 0.41166
batch 027 / 035 | loss: 0.82017 | err: 0.41609
batch 028 / 035 | loss: 0.82002 | err: 0.41574
batch 029 / 035 | loss: 0.82197 | err: 0.41864
batch 030 / 035 | loss: 0.82127 | err: 0.41667
batch 031 / 035 | loss: 0.81444 | err: 0.40927
batch 032 / 035 | loss: 0.81473 | err: 0.40869
batch 033 / 035 | loss: 0.81896 | err: 0.41098
batch 034 / 0

batch 003 / 041 | loss: 1.01440 | err: 0.49479
batch 004 / 041 | loss: 0.95308 | err: 0.46875
batch 005 / 041 | loss: 0.89711 | err: 0.44688
batch 006 / 041 | loss: 0.87044 | err: 0.43359
batch 007 / 041 | loss: 0.84815 | err: 0.42522
batch 008 / 041 | loss: 0.81817 | err: 0.40820
batch 009 / 041 | loss: 0.78893 | err: 0.39236
batch 010 / 041 | loss: 0.77709 | err: 0.38750
batch 011 / 041 | loss: 0.74234 | err: 0.37074
batch 012 / 041 | loss: 0.72970 | err: 0.36393
batch 013 / 041 | loss: 0.71694 | err: 0.35757
batch 014 / 041 | loss: 0.70777 | err: 0.35212
batch 015 / 041 | loss: 0.70169 | err: 0.34844
batch 016 / 041 | loss: 0.69339 | err: 0.34375
batch 017 / 041 | loss: 0.67872 | err: 0.33594
batch 018 / 041 | loss: 0.66236 | err: 0.32769
batch 019 / 041 | loss: 0.64712 | err: 0.32031
batch 020 / 041 | loss: 0.63650 | err: 0.31523
batch 021 / 041 | loss: 0.62527 | err: 0.30915
batch 022 / 041 | loss: 0.61570 | err: 0.30433
batch 023 / 041 | loss: 0.62262 | err: 0.30808
batch 024 / 0

batch 007 / 041 | loss: 0.82098 | err: 0.44531
batch 008 / 041 | loss: 0.82943 | err: 0.45410
batch 009 / 041 | loss: 0.83418 | err: 0.45920
batch 010 / 041 | loss: 0.83296 | err: 0.45859
batch 011 / 041 | loss: 0.83323 | err: 0.45952
batch 012 / 041 | loss: 0.81753 | err: 0.45052
batch 013 / 041 | loss: 0.81932 | err: 0.45192
batch 014 / 041 | loss: 0.81062 | err: 0.44587
batch 015 / 041 | loss: 0.81725 | err: 0.44844
batch 016 / 041 | loss: 0.82211 | err: 0.45166
batch 017 / 041 | loss: 0.81792 | err: 0.44853
batch 018 / 041 | loss: 0.81525 | err: 0.44618
batch 019 / 041 | loss: 0.81946 | err: 0.44860
batch 020 / 041 | loss: 0.81632 | err: 0.44570
batch 021 / 041 | loss: 0.81119 | err: 0.44234
batch 022 / 041 | loss: 0.81636 | err: 0.44460
batch 023 / 041 | loss: 0.81897 | err: 0.44599
batch 024 / 041 | loss: 0.81419 | err: 0.44303
batch 025 / 041 | loss: 0.81152 | err: 0.44156
batch 026 / 041 | loss: 0.80898 | err: 0.43990
batch 027 / 041 | loss: 0.81282 | err: 0.44271
batch 028 / 0